# Import the modules

In [1]:
# import the general modules
from optlang.symbolics import Zero
import cobra
import cplex
import sys

# import kbase
import os
os.environ["HOME"] = 'C:\\Users\\Andrew Freiburger\\Dropbox\\My PC (DESKTOP-M302P50)\\Documents\\UVic Civil Engineering\\Internships\\Agronne\\cobrakbase'
import cobrakbase
token = 'H6SZTTVQGYLKE55YLU5AOZ24OJWLKZNW'
kbase = cobrakbase.KBaseAPI(token)

# define the example individual model and associated API media package
model = kbase.get_from_ws('e_coli_core.kb', 94253)
#model = kbase.get_from_ws('E_iAH991V2', 94253)
model.solver = 'optlang-cplex'

# local ModelSEED database path
modelseed_db_path = '..\..\..\\ModelSEEDDatabase'

cobrakbase 0.2.8


# SimpleThermo dgbin parameterization

## Constrain the model sans dgbin

In [2]:
# ensure that all charges are integers
model = kbase.get_from_ws('e_coli_core.kb', 94253)
for metabolite in model.metabolites:
    try:
        metabolite.charge = int(metabolite.charge)
    except:
        print(metabolite, metabolite.charge)
        metabolite.charge = None
        
# apply the simple thermodynamic constraints 
from modelseedpy.fbapkg import simplethermopkg
stp = simplethermopkg.SimpleThermoPkg(model)
stp.build_package({"dgbin": False})

# execute the model
print(model.objective)
model.summary()

Minimize
-59.81*cpd00001_c0_potential - 59.81*cpd00002_c0_potential - 3.547*cpd00003_c0_potential + 3.547*cpd00004_c0_potential - 13.0279*cpd00005_c0_potential + 13.0279*cpd00006_c0_potential + 59.81*cpd00008_c0_potential + 59.81*cpd00009_c0_potential + 3.7478*cpd00010_c0_potential - 2.8328*cpd00020_c0_potential - 3.7478*cpd00022_c0_potential - 4.9414*cpd00023_c0_potential + 4.1182*cpd00024_c0_potential - 1.7867*cpd00032_c0_potential - 0.2557*cpd00053_c0_potential - 0.5191*cpd00061_c0_potential + 59.81*cpd00067_c0_potential - 0.0709*cpd00072_c0_potential - 0.205*cpd00079_c0_potential - 0.8977*cpd00101_c0_potential - 0.129*cpd00102_c0_potential - 1.496*cpd00169_c0_potential - 0.361*cpd00236_c0_potential


Metabolite,Reaction,Flux,C-Number,C-Flux
Metabolite,Reaction,Flux,C-Number,C-Flux


## Constrain the model with dgbin

In [3]:
# ensure that all charges are integers
model = kbase.get_from_ws('e_coli_core.kb', 94253)
stp.clear()
for metabolite in model.metabolites:
    try:
        metabolite.charge = int(metabolite.charge)
    except:
        print(metabolite, metabolite.charge)
        metabolite.charge = None
        
# apply the simple thermodynamic constraints 
stp = simplethermopkg.SimpleThermoPkg(model)
stp.build_package({"dgbin": True})

# export the constrained LP file 
with open('SimpleThermo_dgbin.lp', 'w') as out:
    out.write(str(model.solver))
    
# execute the model
print(model.objective)
model.summary()

optimal 	 rxn00549_c0


SolverError: CPLEX Error  1217: No solution exists.

# FullThermo infeasibility parameterization

# Brainstorming

In [ ]:
for reaction in model.reactions:
    display(model.solver.status)
    stp.variables['dgbinF'][reaction.id]
dir(stp.variables['dgbinF'][reaction.id].primal)